In [ ]:
import numpy as np
from utils import *
import matplotlib.pyplot as plt
from xfoil import XFoil
from xfoil.model import Airfoil

LAMBDA = 5
perf_bl, R_bl = cal_baseline(lamda=LAMBDA)

In [ ]:
perf_bl, R_bl

In [ ]:
import numpy as np
from utils import *
points = 'results/airfoilPPO.dat'
airfoil = np.loadtxt(points, skiprows=1)
af, R, a, b, perf, cd, CD_BL = lowestD(airfoil, lamda = LAMBDA)
name = points.split('/')[1].split('.')[0]+f'_{a}_{b}F'
np.savetxt(points.split('.')[0]+f'_{a}_{b}F'+'.dat', af, header=points.split('/')[1].split('.')[0]+f'_{a}_{b}F', comments="")

R, a, b, perf, cd, CD_BL

In [ ]:
show_airfoil(af)
detect_intersect(af), cal_thickness(af)

In [ ]:
points = 'samples/Airfoils1D_003.dat'
airfoil = np.loadtxt(points, skiprows=1)
af = setupflap(airfoil, theta=-2, pose=0.65)
evalpreset(af)
# af = setflap(af, theta=2, pose = 0.7)
# evalperf(af)
# show_airfoil(af)

In [ ]:
points = 'samples/Airfoils2D_009_-3.0_0.65F.dat'
airfoil = np.loadtxt(points, skiprows=1)
cal_thickness(airfoil)